# Python import

In [1]:
import cv2 as cv
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image
import cv2
import numpy as np
import matplotlib.pyplot as plt
import torch
import cv2
from scipy.spatial.distance import cdist
from sympy import Point, Polygon, Line
import chess
import chess.svg
from IPython.display import display, SVG
from cairosvg import svg2png
from io import BytesIO
import cv2
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
from pytesseract import Output
import collections


# Image preprocessing

In [2]:
def apply_transformation(image) :
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 0)
    _, otsu_thresholded = cv2.threshold(blurred_image, 165, 255, cv2.THRESH_BINARY)
    edges_image = cv2.Canny(otsu_thresholded, threshold1=50, threshold2=150)
    dilation_kernel = np.ones((3, 3), np.uint8)
    dilated_image = cv2.dilate(edges_image, dilation_kernel, iterations=1)
    lines = cv2.HoughLinesP(dilated_image, 1, np.pi/180, threshold=200, minLineLength=100, maxLineGap=25)
    hough_image = dilated_image
    if lines is not None:
        for i, line in enumerate(lines):
            x1, y1, x2, y2 = line[0]
            cv2.line(hough_image, (x1, y1), (x2, y2), (255,255,255), 2)

    erosion_kernel = np.ones((3, 3), np.uint8)
    erosed_image = cv2.erode(hough_image, erosion_kernel, iterations=1)
    return erosed_image

In [3]:
def get_red_line(image) :
    lines = cv2.HoughLines(image, rho=1, theta=np.pi/180, threshold=250)
    output_lines = []
    if lines is not None:
        for line in lines:
            rho, theta = line[0]
            a = np.cos(theta)
            b = np.sin(theta)
            x0 = a * rho
            y0 = b * rho
            x1 = int(x0 + 2000 * (-b))
            y1 = int(y0 + 2000 * (a))
            x2 = int(x0 - 2000 * (-b))
            y2 = int(y0 - 2000 * (a))
            output_lines.append([x1,y1,x2,y2])
    return output_lines

In [4]:
def put_red_lines(image, lines) :
    if lines is not None:
        for line in lines:
            x1,y1,x2,y2 = line
            cv2.line(image, (x1, y1), (x2, y2), (255, 0, 0), 2)
    return image

In [37]:
def get_raw_oriented_line(image) : 
    lines = cv2.HoughLines(image, rho=1, theta=np.pi/180, threshold=300)
    horizontal_lines = []
    vertical_lines = []
    if lines is not None:
        for line in lines:
            rho, theta = line[0]
            if np.pi / 2 - 0.03 <= theta <= np.pi / 2 + 0.06:
                horizontal_lines.append((rho, theta))
            elif theta <= 0.06 or theta >= np.pi - 0.06:
                vertical_lines.append((rho, theta))
    return (horizontal_lines, vertical_lines)

In [6]:
def get_oriented_line(image) :
    horizontal_lines, vertical_lines = get_raw_oriented_line(image)
    output_horizontal_lines = []
    output_vertical_ilnes = []
    # Draw horizontal lines
    for rho, theta in horizontal_lines:
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a * rho
        y0 = b * rho
        x1 = int(x0 + 2000 * (-b))
        y1 = int(y0 + 2000 * (a))
        x2 = int(x0 - 2000 * (-b))
        y2 = int(y0 - 2000 * (a))
        output_horizontal_lines.append([x1,y1,x2,y2])
    # Draw vertical lines
    for rho, theta in vertical_lines:
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a * rho
        y0 = b * rho
        x1 = int(x0 + 2000 * (-b))
        y1 = int(y0 + 2000 * (a))
        x2 = int(x0 - 2000 * (-b))
        y2 = int(y0 - 2000 * (a))
        output_vertical_ilnes.append([x1,y1,x2,y2])
    return output_horizontal_lines, output_vertical_ilnes


In [7]:
def put_oriented_line(image, horizontal_lines, vertical_lines) :
    for h_line in horizontal_lines:
        x1, y1, x2, y2 = h_line
        cv2.line(image, (x1, y1), (x2, y2), (0, 0, 255), 2)
    for v_line in vertical_lines:
        x1, y1, x2, y2 = v_line
        cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
    return image


In [31]:
def get_intersection_point(image) : 
    horizontal_lines, vertical_lines = get_raw_oriented_line(image)
    intersection_points = []

    for rho_h, theta_h in horizontal_lines:
        for rho_v, theta_v in vertical_lines:
            a_h = np.cos(theta_h)
            b_h = np.sin(theta_h)
            a_v = np.cos(theta_v)
            b_v = np.sin(theta_v)
            determinant = a_h * b_v - b_h * a_v
            if abs(determinant) > 1e-10: 
                x = (b_v * rho_h - b_h * rho_v) / determinant
                y = (a_h * rho_v - a_v * rho_h) / determinant
                intersection_points.append((int(x), int(y)))
    return intersection_points

In [9]:
def put_intersection_point(image,intersection_points) :
    for x, y in intersection_points:
        cv2.circle(image, (x, y), 3, (0, 0, 255), -1)
    return image

In [58]:
def get_corner(image, intersection_points) :
    red_regions = cv2.cvtColor(put_intersection_point(image * 0, intersection_points), cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(red_regions, cv2.COLOR_BGR2GRAY)
    gray = np.float32(gray)
    dst = cv2.cornerHarris(gray, blockSize=3, ksize=3, k=0.06)
    threshold = 0.003 * dst.max()
    corners = np.argwhere(dst > threshold)
    height, width = gray.shape
    image_corners = [
        (0, 0),               # Top-left
        (0, width - 1),       # Top-right 0, width - 1
        (height - 1, 0),      # Bottom-left height - 1, 0
        (height - 1, width - 1)  # Bottom-right
    ]
    refined_corners = []
    for corner in image_corners:
        distances = np.linalg.norm(corners - np.array(corner), axis=1)
        closest_idx = np.argmin(distances)
        refined_corners.append([corners[closest_idx][1],corners[closest_idx][0]])
    
    return refined_corners


In [11]:
def apply_crop_image(image, corners) :
    corners = np.float32(corners)
    src_points = corners
    dst_points = np.float32([
        [0, 0],
        [500, 0],
        [0, 500],
        [500, 500]
    ])
    matrix = cv2.getPerspectiveTransform(src_points, dst_points)
    output_size = (500, 500) 
    warped_image = cv2.warpPerspective(image, matrix, output_size)
    
    return warped_image

In [12]:
def apply_sol2(image) :
    transformed_image = apply_transformation(image)
    red_lines = get_red_line(transformed_image)
    horizontal_lines, vertical_lines = get_oriented_line(transformed_image)
    intersection_points = get_intersection_point(transformed_image)
    corners = get_corner(image,intersection_points)
    croped_image = apply_crop_image(image,corners)
    plt.imshow(put_oriented_line(image,horizontal_lines,vertical_lines))

In [13]:
def line_intersection(_line1, _line2):
    line1 = (_line1[0:2],_line1[2:4])
    line2 = (_line2[0:2],_line2[2:4])
    xdiff = (line1[0][0] - line1[1][0], line2[0][0] - line2[1][0])
    ydiff = (line1[0][1] - line1[1][1], line2[0][1] - line2[1][1])

    def det(a, b):
        return a[0] * b[1] - a[1] * b[0]

    div = det(xdiff, ydiff)
    if div == 0:
       return -1, -1

    d = (det(*line1), det(*line2))
    x = det(d, xdiff) / div
    y = det(d, ydiff) / div
    return x, y

def filter_separate_lines(lines):
    separate_lines = []
    for i, line1 in enumerate(lines):
        is_separate_from_all = True
        for j, line2 in enumerate(separate_lines):
            x, y = line_intersection(line1,line2)
            if x >= 0 and x <= 1200 and y >= 300 and y <= 1800:
                is_separate_from_all = False
                break
            if(np.abs(line1[2] - line1[0]) < np.abs(line1[3] - line1[1])) :
                diff1 = (line1[2] - line1[0]) / (line1[3] - line1[1])
                diff2 = (line2[2] - line2[0]) / (line2[3] - line2[1])
                x1 = (500.0 - line1[1]) * diff1 + line1[0]
                x2 = (500.0 - line2[1]) * diff2 + line2[0]
                if(np.abs(x2-x1)) < 10 :
                    is_separate_from_all = False
                    break
            else :
                diff1 = (line1[3] - line1[1]) / (line1[2] - line1[0])
                diff2 = (line2[3] - line2[1]) / (line2[2] - line2[0])
                y1 = (1000.0 - line1[0]) * diff1 + line1[1]
                y2 = (1000.0 - line2[0]) * diff2 + line2[1]
                if(np.abs(y1-y2) < 10) :
                    is_separate_from_all = False
                    break
        if is_separate_from_all:
            separate_lines.append(line1)
    return separate_lines

In [14]:
def get_intersect_board_line(lines,corners) :
    output_lines = []
    for line in lines :
        board_poly = Polygon(corners[0], corners[1], corners[3], corners[2])
        if board_poly.intersection(Line(Point(line[0],line[1]), Point(line[2], line[3]))) :
            output_lines.append(line)
    return output_lines

In [15]:
def get_shrink_corners(corners, x) :
    corners[0][0] += x
    corners[0][1] += x
    corners[1][0] -= x
    corners[1][1] += x
    corners[2][0] -= x
    corners[2][1] -= x
    corners[3][0] += x
    corners[3][1] -= x
    return corners

In [16]:
def get_app_board_size(corners) :
    return  (((corners[3][1] - corners[0][1]) ** 2 + (corners[3][0] - corners[0][0]) ** 2) ** 1/2 + \
    ((corners[2][1] - corners[1][1]) ** 2 + (corners[2][0] - corners[1][0]) ** 2) ** 1/2) / 2

In [40]:
def get_cell_cordinate(image, corners = None, horizontal_lines = None, vertical_lines = None) :
    if corners == None :
        transformed_image = apply_transformation(image)
        intersection_points = get_intersection_point(transformed_image)
        corners = get_corner(image,intersection_points)
    # board_size = int(get_app_board_size(corners))
    shrink_corners = get_shrink_corners([corners[0],corners[1],corners[3],corners[2]], int(120))
    board_mask = cv2.fillPoly(image * 0,[np.array(shrink_corners)],(255,255,255))
    # mask_image = cv2.bitwise_and(image,board_mask)
    if(horizontal_lines == None or vertical_lines == None) :
        transformed_image = apply_transformation(image)
        horizontal_lines, vertical_lines = get_oriented_line(transformed_image)
    horizontal_lines_separate = filter_separate_lines(horizontal_lines)
    vertical_lines_separate = filter_separate_lines(vertical_lines)
    horizontal_lines_intersect = get_intersect_board_line(horizontal_lines_separate, shrink_corners)
    vertical_lines_intersect = get_intersect_board_line(vertical_lines_separate, shrink_corners)

    return horizontal_lines_intersect, vertical_lines_intersect

# Chess piece detection

In [211]:
MODEL_FOLDER = "yolov5/"
VIDEO_FOLDER = "yolov5/test-video/"
OUTPUT_FOLDER = ""

MODEL_FILE = "chess-v10.pt"
VIDEO_FILE = "4_Move_studet.mp4"
OUTPUT_FILE = "detectAlgoOutput.avi"

In [212]:
def put_piece(board, data, horizontal_lines_intersect, vertical_lines_intersect) :
    #print("uuuuuuuuuuuuuuuuuu",len(horizontal_lines_intersect),len(vertical_lines_intersect))
    def is_point_right(p, l):
        x, y = p
        x1, y1, x2, y2 = l
        diff = (x2 - x1) / (y2 - y1)
        pos_x = (y - y1) * diff + x1
        #if data["name"] == "w-rook" :
        #    print(pos_x,x)
        return pos_x < x
    def is_point_above(p, l):
        x, y = p
        x1, y1, x2, y2 = l
        diff = (y2 - y1) / (x2 - x1)
        pos_y = (x - x1) * diff + y1
        return pos_y > y
    h_index = 0
    v_index = 0
    center = ((data["xmax"] + data["xmin"]) / 2, (data["ymax"] * 0.9 + data["ymin"] * 0.1))
    for l in vertical_lines_intersect :
        if is_point_right(center, l) :
            h_index = h_index + 1
    for l in horizontal_lines_intersect :
        if is_point_above(center, l) :
            v_index = v_index + 1

    v_index -= len(horizontal_lines_intersect) - 7
    h_index -= len(vertical_lines_intersect) - 7
    if(v_index < 0 or h_index < 0) :
        return board
    if(board[v_index][h_index] == '-') :
        board[v_index][h_index] = data["name"]
    
    return board

In [213]:
def re_roate(image) :
    count = 0
    transformed_image = apply_transformation(image)
    intersection_points = get_intersection_point(transformed_image)
    corners = get_corner(image,intersection_points)
    board_size = int(get_app_board_size(corners))
    shrink_corners = get_shrink_corners([corners[0],corners[1],corners[3],corners[2]], int(board_size * 0.0000112))
    image = apply_crop_image(image,corners)
    while(count < 5):
        height = image.shape[0]
        width = image.shape[1]
        plt.imshow(image)
        plt.savefig("output4.jpg")

        bottom_left = image[int(height * 0.95):int(height*0.99), int(width*0.00):int(width*0.2)]
        bottom_right = image[int(height * 0.95):int(height*0.99), int(width*0.80):int(width*1.0)]

        plt.imshow(bottom_left)
        plt.show()

        bottom_left_string = pytesseract.image_to_string(bottom_left, config="--psm 10").strip()
        bottom_right_string = pytesseract.image_to_string(bottom_right, config="--psm 10").strip()

        print(bottom_left_string,bottom_right_string)


        if ('h' in bottom_left_string and 'a' in bottom_right_string):
            break
        else:
            image = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
        count = count + 1
        
    return count

In [214]:
def get_chess_board(board_string) :
    board = chess.Board.empty()
    for i,t1 in enumerate(("1","2","3","4","5","6","7","8")) :
        for j,t2 in enumerate(("h","g","f","e","d","c","b","a")) :
            if(board_string[i][j] == '-') :
                continue
            elif (board_string[i][j] == 'w-king') :
                board.set_piece_at(chess.parse_square(t2 + t1),chess.Piece(chess.KING, chess.WHITE))
            elif (board_string[i][j] == 'w-queen') :
                board.set_piece_at(chess.parse_square(t2 + t1),chess.Piece(chess.QUEEN, chess.WHITE))
            elif (board_string[i][j] == 'w-bishop') :
                board.set_piece_at(chess.parse_square(t2 + t1),chess.Piece(chess.BISHOP, chess.WHITE))
            elif (board_string[i][j] == 'w-knight') :
                board.set_piece_at(chess.parse_square(t2 + t1),chess.Piece(chess.KNIGHT, chess.WHITE))
            elif (board_string[i][j] == 'w-rook') :
                board.set_piece_at(chess.parse_square(t2 + t1),chess.Piece(chess.ROOK, chess.WHITE))
            elif (board_string[i][j] == 'w-pawn') :
                board.set_piece_at(chess.parse_square(t2 + t1),chess.Piece(chess.PAWN, chess.WHITE))
            elif (board_string[i][j] == 'b-king') :
                board.set_piece_at(chess.parse_square(t2 + t1),chess.Piece(chess.KING, chess.BLACK))
            elif (board_string[i][j] == 'b-queen') :
                board.set_piece_at(chess.parse_square(t2 + t1),chess.Piece(chess.QUEEN, chess.BLACK))
            elif (board_string[i][j] == 'b-bishop') :
                board.set_piece_at(chess.parse_square(t2 + t1),chess.Piece(chess.BISHOP, chess.BLACK))
            elif (board_string[i][j] == 'b-knight') :
                board.set_piece_at(chess.parse_square(t2 + t1),chess.Piece(chess.KNIGHT, chess.BLACK))
            elif (board_string[i][j] == 'b-rook') :
                board.set_piece_at(chess.parse_square(t2 + t1),chess.Piece(chess.ROOK, chess.BLACK))
            elif (board_string[i][j] == 'b-pawn') :
                board.set_piece_at(chess.parse_square(t2 + t1),chess.Piece(chess.PAWN, chess.BLACK))
    return board         
            

In [215]:
def find_move(board1: chess.Board, board2: chess.Board) -> chess.Move:
    for move in board1.legal_moves:
        temp_board = board1.copy()
        temp_board.push(move)
        if temp_board.board_fen() == board2.board_fen():
            return move
    return None

In [216]:
def write_board(file_name, board) : 
    with open(file_name,"w") as f :
        f.write(chess.svg.board(board=board))

In [217]:
def calculate_iou(box1, box2):
    x1, y1, x2, y2 = box1
    x1_, y1_, x2_, y2_ = box2

    xi1, yi1 = max(x1, x1_), max(y1, y1_)
    xi2, yi2 = min(x2, x2_), min(y2, y2_)
    inter_area = max(0, xi2 - xi1 + 1) * max(0, yi2 - yi1 + 1)

    box1_area = (x2 - x1 + 1) * (y2 - y1 + 1)
    box2_area = (x2_ - x1_ + 1) * (y2_ - y1_ + 1)

    union_area = box1_area + box2_area - inter_area
    return inter_area / union_area

In [218]:
def non_max_suppression_custom(detections, iou_threshold=0.6):
    filtered_boxes = []
    while len(detections) > 0:
        detections = sorted(detections, key=lambda x: x['confidence'], reverse=True)
        best_box = detections.pop(0) 
        filtered_boxes.append(best_box)

        detections = [
            box for box in detections
            if calculate_iou(
                (best_box['xmin'], best_box['ymin'], best_box['xmax'], best_box['ymax']),
                (box['xmin'], box['ymin'], box['xmax'], box['ymax'])
            ) < iou_threshold
        ]
    return filtered_boxes

In [219]:
def load_chess_model() :
    model = torch.hub.load('ultralytics/yolov5', 'custom', path=MODEL_FOLDER + MODEL_FILE)
    model.conf = 0.3
    model.iou = 0.9
    return model

In [220]:
def load_coco_model() :
    model = torch.hub.load('ultralytics/yolov5', 'yolov5l') 
    model.conf = 0.2
    return model

In [221]:
def get_frame_predict(frame, model) :
    # model.to()
    results = model(frame)
    detections = results.pandas().xyxy[0]
    detections_list = [
        {
            'xmin': int(row['xmin']),
            'ymin': int(row['ymin']),
            'xmax': int(row['xmax']),
            'ymax': int(row['ymax']),
            'confidence': float(row['confidence']),
            'name': row['name']
        }
        for _, row in detections.iterrows()
    ]
    filtered_detections = non_max_suppression_custom(detections_list, iou_threshold=0.6)
    return filtered_detections

In [222]:
import warnings
warnings.filterwarnings("ignore")  # Ignore all warnings globally

In [224]:
chess_model = load_chess_model()
coco_model = load_coco_model() 

video_path = VIDEO_FOLDER + VIDEO_FILE
output_path = OUTPUT_FOLDER + OUTPUT_FILE

cap = cv2.VideoCapture(video_path)
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(output_path, fourcc, 1, (width, height))

frame_count = 0
processed_frame = 0

board_state = []
hand_on_last_frame = True

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    if frame_count % fps == 0:
        processed_frame += 1
        original_frame = frame.copy()
        print(f"Processing frame {frame_count} (Frame {processed_frame} at 1 FPS) :")

        coco_results = coco_model(frame)
        coco_detections = coco_results.pandas().xyxy[0]
        centers = []

        person_detections = coco_detections[coco_detections['name'] == 'person']
        if not person_detections.empty:
            hand_on_last_frame = True
            for _, person in person_detections.iterrows():
                xmin, ymin, xmax, ymax = int(person['xmin']), int(person['ymin']), int(person['xmax']), int(person['ymax'])
                label = f"Person {person['confidence']:.2f}"
                cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (255, 0, 0), 2)  # Blue for person
                cv2.putText(frame, label, (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
        else:
            h_cell_line, v_cell_line = get_cell_cordinate(frame)
            if(hand_on_last_frame) :
                board_state.append([])
            hand_on_last_frame = False 
            chess_results = chess_model(frame)
            chess_detections = chess_results.pandas().xyxy[0]
            chess_detections_list = [
                {
                    'xmin': int(row['xmin']),
                    'ymin': int(row['ymin']),
                    'xmax': int(row['xmax']),
                    'ymax': int(row['ymax']),
                    'confidence': float(row['confidence']),
                    'name': row['name']
                }
                for _, row in chess_detections.iterrows()
            ]
            filtered_chess_detections = non_max_suppression_custom(chess_detections_list)

            cur_board = [["-"]*8,["-"]*8,["-"]*8,["-"]*8,["-"]*8,["-"]*8,["-"]*8,["-"]*8]
            
            for det in filtered_chess_detections:
                xmin, ymin, xmax, ymax = det['xmin'], det['ymin'], det['xmax'], det['ymax']
                label = f"{det['name']} {det['confidence']:.2f}"
                color = (0, 255, 0) if det['name'] == "chessboard" else (0, 0, 255)
                cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), color, 2)
                cv2.putText(frame, label, (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
                put_piece(cur_board,det,h_cell_line,v_cell_line)
                center = (int(det["xmax"] + det["xmin"]) // 2, int(det["ymax"] * 0.9 + det["ymin"] * 0.1))
                centers.append(center)
            if(len(h_cell_line) == 7 and len(v_cell_line) == 7) :
                board_state[-1].append(get_chess_board(cur_board))
            elif(len(board_state[-1]) > 0) :
                board_state[-1].append(board_state[-1][-1])
            #print(get_chess_board(cur_board))

        transformed_image = apply_transformation(original_frame)
        horizontal_lines, vertical_lines = get_oriented_line(transformed_image)
        horizontal_lines_separate = filter_separate_lines(horizontal_lines)
        vertical_lines_separate = filter_separate_lines(vertical_lines)
        intersection_points = get_intersection_point(transformed_image)
        corners = get_corner(original_frame,intersection_points)
        # board_size = int(get_app_board_size(corners))
        shrink_corners = get_shrink_corners([corners[0],corners[1],corners[3],corners[2]], int(120))
        board_mask = cv2.fillPoly(original_frame * 0,[np.array(shrink_corners)],(255,255,255))

        horizontal_lines_intersect = get_intersect_board_line(horizontal_lines_separate, shrink_corners)
        vertical_lines_intersect = get_intersect_board_line(vertical_lines_separate, shrink_corners)
        
        # frame = cv2.bitwise_and(frame,board_mask)
        frame = put_oriented_line(frame,horizontal_lines_intersect,vertical_lines_intersect)

        frame = put_intersection_point(frame,centers)
        out.write(frame)

        cv2.imshow('YOLOv5 Chess Detection with Person Detection', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    frame_count += 1

cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Video with detections saved to {output_path}")

Using cache found in C:\Users\asus/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-12-9 Python-3.12.4 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)

Fusing layers... 
Model summary: 212 layers, 20897385 parameters, 0 gradients, 48.0 GFLOPs
Adding AutoShape... 
Using cache found in C:\Users\asus/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-12-9 Python-3.12.4 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)

Fusing layers... 
YOLOv5l summary: 367 layers, 46533693 parameters, 0 gradients, 109.0 GFLOPs
Adding AutoShape... 


Processing frame 0 (Frame 1 at 1 FPS) :
Processing frame 29 (Frame 2 at 1 FPS) :
Processing frame 58 (Frame 3 at 1 FPS) :
Processing frame 87 (Frame 4 at 1 FPS) :
Processing frame 116 (Frame 5 at 1 FPS) :
Processing frame 145 (Frame 6 at 1 FPS) :
Processing frame 174 (Frame 7 at 1 FPS) :
Processing frame 203 (Frame 8 at 1 FPS) :
Processing frame 232 (Frame 9 at 1 FPS) :
Processing frame 261 (Frame 10 at 1 FPS) :
Processing frame 290 (Frame 11 at 1 FPS) :
Processing frame 319 (Frame 12 at 1 FPS) :
Processing frame 348 (Frame 13 at 1 FPS) :
Processing frame 377 (Frame 14 at 1 FPS) :
Processing frame 406 (Frame 15 at 1 FPS) :
Processing frame 435 (Frame 16 at 1 FPS) :
Processing frame 464 (Frame 17 at 1 FPS) :
Processing frame 493 (Frame 18 at 1 FPS) :
Processing frame 522 (Frame 19 at 1 FPS) :
Processing frame 551 (Frame 20 at 1 FPS) :
Processing frame 580 (Frame 21 at 1 FPS) :
Processing frame 609 (Frame 22 at 1 FPS) :
Processing frame 638 (Frame 23 at 1 FPS) :
Processing frame 667 (Fra

In [225]:
def most_frequent_pieces(chess_boards):
    piece_count = [[collections.Counter() for _ in range(8)] for _ in range(8)]
    for board in chess_boards:
        for square in chess.SQUARES:
            piece = board.piece_at(square)
            if piece:
                row, col = divmod(square, 8)
                piece_count[row][col][piece.symbol()] += 1
    result_board = chess.Board.empty()
    for i in range(8):
        for j in range(8):
            most_common_piece = piece_count[i][j].most_common(1)
            if most_common_piece:
                piece_symbol = most_common_piece[0][0]
                square = chess.square(j, 7-i) 
                result_board.set_piece_at(square, chess.Piece.from_symbol(piece_symbol))
    return result_board

In [226]:
def find_move(board1, board2):
    move_from = None
    move_to = None
    moved_piece = None
    for square in chess.SQUARES:
        piece1 = board1.piece_at(square)
        piece2 = board2.piece_at(square)
        if piece1 and not piece2:
            moved_piece = piece1
            move_from = square
        if not piece1 and piece2:
            moved_piece = piece2
            move_to = square
    if move_from is not None and move_to is not None:
        move = chess.Move(move_from, move_to)
        return moved_piece.symbol(), move.uci() 
    if move_from is not None :
        for square in chess.SQUARES:
            piece1 = board1.piece_at(square)
            piece2 = board2.piece_at(square)
            if piece1 and piece2 and piece1 != piece2 and piece2 == moved_piece:
                moved_piece = piece2
                move_to = square
                move = chess.Move(move_from, move_to)
                return moved_piece.symbol(), move.uci()[0:2] + "x" + move.uci()[2:4]
        for square in chess.SQUARES:
            piece1 = board1.piece_at(square)
            piece2 = board2.piece_at(square)
            if piece1 and piece2 and piece1 != piece2:
                moved_piece = piece2
                move_to = square
                move = chess.Move(move_from, move_to)
                return moved_piece.symbol(), move.uci()[0:2] + "x" + move.uci()[2:4]
    return None 

In [227]:
def pgn(input):
    if(input[0][0] == input[0][0].lower()):
        output = ['..']
    else:
        output = []
    for i in range(len(input)):
        if(input[i][0] == 'p'):
            if(input[i][1][2] == 'x'):
                output.append(input[i][1][-3:])
            else:
                output.append(input[i][1][-2:])
        else:
            if(input[i][1][2] == 'x'):
                output.append(input[i][0] + input[i][1][-3:])
            else:
                output.append(input[i][0] + input[i][1][-2:])
    result = ""
    count = 1
    for i in range(len(output)):
        if(i % 2 == 0):
            result = result + str(count) + ". " + output[i].lower() + " "
            count = count + 1
        else:
            result = result + output[i].lower() + " "
        
    return result.strip()

In [228]:
board_state_stable = []
for i, b in enumerate(board_state) :
    print(len(b))
    if(i != 0 and len(b) < 5) :
        continue
    if(i != 0) :
        board_state_stable.append(most_frequent_pieces(b[2:]))
    else :
        board_state_stable.append(most_frequent_pieces(b))
for b in board_state_stable :
    print(b)
    print()
moves = []
for i in range(len(board_state_stable) - 1) :
    moves.append(find_move(board_state_stable[i],board_state_stable[i + 1]))
    if(moves[-1] == None) :
        moves.pop()
        continue
    print(moves[-1])

print(pgn(moves))

4
32
41
1
0
6
9
r . . q q n n r
r p p b . . r .
. . . p . p . .
n . . P p . . .
. P . . P . . N
. . N . . . B .
P . P . . P P P
R . . Q K B . R

r . . q q n n r
r p p b . . r .
. . . p . . . .
n . . P p . . .
. P . . P p . N
. . N . . . B .
P . P . . P P P
R . . Q K B . R

r . . q q n n r
r p p b . . r .
. . . p . . N .
n . . P p . . .
. P . . P p . .
. . N . . . B .
P . P . . P P P
R . . Q K B . R

r . . q q n n r
r p p b . . r .
. . . p . . N .
n . . P p . . .
. P . . P . . .
. . N . . . p .
P . P . . P P P
R . . Q K B . R

r . . q q n n N
r p p b . . r .
. . . p . . . .
n . . P p . . .
. P . . P . . .
. . N . . . p .
P . P . . P P P
R . . Q K B . R

('p', 'f6f4')
('N', 'h4g6')
('p', 'f4xg3')
('N', 'g6xh8')
1. .. f4 2. ng6 xg3 3. nxh8
